In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import datetime as tm
import json
import pandas as pd
from pyspark.storagelevel import StorageLevel
from pyspark.sql import *
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext, HiveContext
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import cPickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
from tr_utils import *

#Téléchargement des cookies du site 750gr sur une semaine
adv = 656228
dates = get_period('2017-06-07', '2017-06-14')
path = 'gs://tr-parquet/its_panel/'
dfs = []
for date in dates:
    dfs += [sqlContext.read.load(path+date+'/ad='+str(adv))]
    
data = reduce(lambda x, y: x.unionAll(y), dfs)


#On récupère les données sous forme d'un DataFrame
dl = data.filter((F.col('ty')=='datalayer')&(F.col('id2')!='')&(F.col('id2')!=-1)&(F.col('id2')!=0))
#creation d'un schema puis application a l'objet dataLayer
test = spark.read.json(dl.limit(1000).rdd.map(lambda x: x.dl))
schema = test.schema
with open('schema_dl_jv.pickle', 'w') as f:
    cPickle.dump(schema, f)
    
dl_parsed = dl.withColumn('dl2', F.from_json('dl', schema))

#Les données les plus importantes sont celles situées dans le dataLayer
#On sélectionne les identifiants des users, les recettes consultées ainsi que les ingrédients associés aux recettes
#Les ingrédients ne sont pas donnés comme des mots clés, mais avec les quantités associées (sous forme de phrase..)
df=dl_parsed.select('dl2.ses.uuid2','dl2.dataLayer.sectionData.averageScore','dl2.dataLayer.sectionData.title'
                   ,'dl2.dataLayer.sectionData.ingredients')



In [2]:
import unicodedata
def strip_accents(s):
    if not s : 
        return s
    return s.encode('ascii', 'ignore').decode('ascii')

In [3]:
strip_accents_udf = F.udf(strip_accents,StringType())

In [ ]:
#On regroupe dans une même ligne toutes les données relatives à un user: recettes et ingrédients associés à ces recettes
from pyspark.sql.functions import collect_list,concat_ws
df2=df.select('uuid2','averageScore',strip_accents_udf('title').alias('title'),strip_accents_udf('ingredients').alias('ingredients')).dropDuplicates().cache()
df3= df2.select('uuid2','averageScore',concat_ws('---', df2.title, df2.ingredients).alias('concat'))
    
grouped = df3.groupBy(['uuid2']).agg(collect_list('concat').alias('concat2'))

#Nous séparons notre base deux: la base train et la base test.
#Sur la base train, nous modéliserons nos 100 régressions logistiques, et évaluerons la pertinence du modèle sur la base test
#Sur la base test, nous ne gardons que 2/3 des recettes, qui nous servent à obtenir les recommendations, et testons 
#nos recommmendations sur le tiers restant.


def lr(u):
    l=[]
    for i in u.concat2:
        l.append(i.split('---')[0])
    return l

def ir(u):
    l=[]
    try:
        for i in u.concat2:
            l.append(i.split('---')[1])
        return l
    except:
        return l

rdd=grouped.rdd.map(lambda x: Row(uuid2=x[0], listerecettes2=lr(x),ingredientsrecettes2=ir(x)))
grouped = sqlContext.createDataFrame(rdd)

def countrecettes(u):
    p=set(u)
    return float(len(list(p)))

countrecettes_=F.udf(countrecettes,FloatType())


def enleverdoublons(u):
    p=set(u)
    return list(p)

enleverdoublons_=F.udf(enleverdoublons,ArrayType(StringType()))


countgrouped=grouped.select('uuid2',enleverdoublons_('listerecettes2').alias('listerecettes3'),enleverdoublons_('ingredientsrecettes2').alias('ingredientsrecettes3'),'listerecettes2')
countgrouped=countgrouped.select('uuid2','listerecettes3','ingredientsrecettes3',countrecettes_('listerecettes3').alias('count'))
#Pour notre base test, nous ne sélectionnons que les individus qui ont visionné plus de 3 recettes, 
#afin d'obtenir les recommendations à l'aide de 2 recettes au minimum, et de vérifier la pertinence des recommendations à l'aide 
#d'une recette au minimum
grouped=countgrouped.where("count >= 3.0")

def trainrecettes(u):
    return u[0:int((2*len(u))/3)]

trainrecettes_=F.udf(trainrecettes,ArrayType(StringType()))

dftrain=grouped.select('uuid2','count',trainrecettes_('ingredientsrecettes3').alias('ingredientsrecettes'),trainrecettes_('listerecettes3').alias('listerecettes'))

#on veut construire une liste contenant tous les mots pertinents de la liste d'ingrédients, en se débarassant 
#dans un premier temps des quantités, puis des stopwords à l'aide de la commande existante du package NLTK.
#On se débarasse également des lettres seules exclues, provenant d'abréviations comme c a s pour 'cuillière à soupe',
#g pour grammes, mg, kg etc.

import re
from nltk.corpus import stopwords
from pyspark.ml.feature import StopWordsRemover

list_nb = '0 1 2 3 4 5 6 7 8 9'.split()
list_lt='a b c d e f g h i j k l m n o p q r s t u v w kg mg gr cl ml dl cuil'.split()
stopwords=StopWordsRemover.loadDefaultStopWords('french')

def transform (l):
    return [mot for recette in l for mot in re.split(' |;',''.join([i if i not in list_nb else '' for i in recette ])) if mot!='' if mot not in stopwords and mot not in list_lt] 

transform_df  = F.udf(transform,ArrayType(StringType()))

#Nouveau dataFrame auquel on a appliqué la fonction transform
dftrain2=dftrain.select('uuid2','listerecettes',transform_df('ingredientsrecettes').alias('ingredientsrecettes2'))


#Nous obtenons ici pour chaque individu un vecteur de la taille du corpus des ingrédients des recettes, dont chaque élément 
#représente la fréquence de l'ingrédient dans les recettes consultées par l'individu
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="ingredientsrecettes2", outputCol="CountVectorizer")

model = cv.fit(dftrain2)

result1 = model.transform(dftrain2)



from pyspark.mllib.linalg import SparseVector

def bigvect1(x):
    return ([int(i) for i in x])


bigvect1_  = F.udf(bigvect1,ArrayType(IntegerType()))

resultat1=result1.select('uuid2','listerecettes','ingredientsrecettes2',bigvect1_('CountVectorizer').alias('CountVectorizer2'))
resultat1.cache().show(truncate=False)

def testrecettes(u):
    return u[int((2*len(u))/3):len(u)]

testrecettes_=F.udf(testrecettes,ArrayType(StringType()))

resultattest=grouped.select('uuid2',testrecettes_('listerecettes3').alias('recettestest'))

In [ ]:
#Nous obtenons ici pour chaque individu un vecteur de la taille du corpus des ingrédients des recettes, dont chaque élément 
#représente le TF-IDF de l'ingrédient dans les recettes consultées par l'individu

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="ingredientsrecettes2", outputCol="features")
model = cv.fit(dftrain2)
results = model.transform(dftrain2)


idf = IDF(inputCol="features", outputCol="tfidf")
idfModel = idf.fit(results)
result2 = idfModel.transform(results)

result2.select("uuid2", "tfidf").show(truncate=False)

from pyspark.mllib.linalg import SparseVector

def bigvect2(x):
    return ([round(i,3) for i in x])



bigvect2_  = F.udf(bigvect2,ArrayType(FloatType()))

resultat2=result2.select('uuid2','listerecettes','ingredientsrecettes2',bigvect2_('tfidf').alias('tfidf2'))

In [6]:
#Recommendations de users à recettes selon la méthode de Jaccard sur les vecteurs des fréquences des mots
def recorecettesjaccard(id,df,resultat1):
    
    #Nous créons les vecteurs des fréquences des ingrédients pour chaque recette de la base train
    from pyspark.sql.functions import collect_list
    grouped_ = df.groupBy(['title']).agg(collect_list("uuid2").alias('users'),collect_list("ingredients").alias('ingredients2'))
    grouped_.head(50)

    def double(l):
        return list(set(l))

    double_=F.udf(double,ArrayType(StringType()))

    grouped_=grouped_.select('title','users','ingredients2',double_('ingredients2').alias('ingredients3'))

    dfid= resultat1.filter(resultat1.uuid2 ==id )
    recettes3=dfid.collect()[0].listerecettes
    ingredients3=dfid.collect()[0].ingredientsrecettes2
    
    import re
    from nltk.corpus import stopwords
    from pyspark.ml.feature import StopWordsRemover

    list_nb = '0 1 2 3 4 5 6 7 8 9'.split()
    list_lt='a b c d e f g h i j k l m n o p q r s t u v w kg mg gr cl ml dl'.split()
    stopwords=StopWordsRemover.loadDefaultStopWords('french')

    def transform2 (l):
         return [mot for recette in l for mot in re.split(' |;',''.join([i if i not in list_nb else '' for i in recette ])) if mot!='' if mot not in stopwords and mot not in list_lt] 

    transform2_df  = F.udf(transform2,ArrayType(StringType()))

    grouped_=grouped_.select('title','users','ingredients3',transform2_df('ingredients3').alias('ingredients4'))

    newRow = spark.createDataFrame([[id,[id],ingredients3,ingredients3]])
    grouped_ = grouped_.union(newRow)

    from pyspark.ml.feature import CountVectorizer

    cv = CountVectorizer(inputCol="ingredients4", outputCol="CountVectorizer")

    model = cv.fit(grouped_)

    result3 = model.transform(grouped_)

    from pyspark.mllib.linalg import SparseVector

    def bigvect3(x):
        return ([int(i) for i in x])


    bigvect3_  = F.udf(bigvect3,ArrayType(IntegerType()))

    resultat3=result3.select('title','users','ingredients4',bigvect3_('CountVectorizer').alias('CountVectorizer2'))

    x=resultat3.filter(resultat3.title==id).collect()[0].CountVectorizer2
    
    #Calcul de l'indice de Jaccard entre le vecteur des fréquences des ingrédients du user en entrée de la fonction 
    #et les vecteurs des fréquences des ingrédients de chaque recette

    def inter(y):
        return sum(i+j for i,j in zip(x,y) if i>0 and j>0)

    def union(y):
        return sum(i+j for i,j in zip(x,y))

    def jaccard(y):
        return round(float(inter(y))/float(union(y)),3)


    jaccard_  = F.udf(jaccard,FloatType())

    resultatjac=resultat3.select('title','users','ingredients4','CountVectorizer2',jaccard_('CountVectorizer2').alias('Jaccard'))

    resultatjac=resultatjac.filter(resultatjac.title !=id).filter(resultatjac.Jaccard !=1.0 )
    for i in recettes3:
        resultatjac=resultatjac.filter(resultatjac.title !=i)

    from pyspark.sql.functions import desc

    resultatjac=resultatjac.sort(desc('Jaccard'))

    #Recommendations des 5 recettes aux indices de Jaccard les plus élevés
    reco=resultatjac.head(5)

    return ([i.title for i in reco])



In [ ]:
identifiants=resultat1.select('uuid2').collect()
identifiants=[i.uuid2 for i in identifiants]

In [ ]:
#Evaluation du modèle de recommendation de users à recettes selon la distance de Jaccard pour les individus de la base
dic={}
for i in identifiants:
    dic.setdefault(i,recorecettesjaccard(i,df2,resultat1))

In [ ]:
dic

In [ ]:
from pyspark.sql import Row
l = [(x,y) for x,y in zip(dic.keys(),dic.values())]
rdd = sc.parallelize(l)
rdd2 = rdd.map(lambda x: Row(uuid2=x[0], recettesreco=x[1]))
dff = sqlContext.createDataFrame(rdd2)

In [ ]:
resultat =resultattest.select('uuid2','recettestest').join(dff,'uuid2')

In [ ]:
#Evaluation de la performance, autrement dit le pourcentage de personnes dont au moins une des 5 recettes recommendées
#appartient aux 1/3 de recettes isolées au début pour chaque individu
def perf(u):
    nb=0.0
    for i in u.recettesreco:
        if i in u.recettestest:
            nb=1.0
            break
    return u.uuid2,u.recettestest,u.recettesreco,nb


l=resultat.rdd.map(lambda row: perf(row)).collect()

    

In [13]:
#Calcul de la performance
a=[i[3] for i in l]
tauxperfjaccard=float(sum(a))/float(len(a))

In [14]:
tauxperfjaccard

0.6486486486486487

In [ ]:
#Recommendations de users à recettes selon la méthode de la similarité cosinus sur les vecteurs des fréquences des mots

def recorecettessimicos(id,df,resultat1):
 
    from pyspark.sql.functions import collect_list
    df=df.dropDuplicates()
    grouped_ = df.groupBy(['title']).agg(collect_list("uuid2").alias('users'),collect_list("ingredients").alias('ingredients2'))


    def double(l):
        return list(set(l))


    double_=F.udf(double,ArrayType(StringType()))

    grouped_=grouped_.select('title','users','ingredients2',double_('ingredients2').alias('ingredients3'))

    dfid= resultat1.filter(resultat1.uuid2 ==id )
    recettes3=dfid.collect()[0].listerecettes
    ingredients3=dfid.collect()[0].ingredientsrecettes2

    import re
    from nltk.corpus import stopwords
    from pyspark.ml.feature import StopWordsRemover

    list_nb = '0 1 2 3 4 5 6 7 8 9'.split()
    list_lt='a b c d e f g h i j k l m n o p q r s t u v w kg mg gr cl ml dl'.split()
    stopwords=StopWordsRemover.loadDefaultStopWords('french')

    def transform2 (l):
         return [mot for recette in l for mot in re.split(' |;',''.join([i if i not in list_nb else '' for i in recette ])) if mot!='' if mot not in stopwords and mot not in list_lt] 

    transform2_df  = F.udf(transform2,ArrayType(StringType()))

    grouped_=grouped_.select('title','users','ingredients3',transform2_df('ingredients3').alias('ingredients4'))

    newRow = spark.createDataFrame([[id,[id],ingredients3,ingredients3]])
    grouped_ = grouped_.union(newRow)

    from pyspark.ml.feature import CountVectorizer

    cv = CountVectorizer(inputCol="ingredients4", outputCol="CountVectorizer")

    model = cv.fit(grouped_)

    result3 = model.transform(grouped_)

    from pyspark.mllib.linalg import SparseVector

    def bigvect3(x):
        return ([int(i) for i in x])


    bigvect3_  = F.udf(bigvect3,ArrayType(IntegerType()))

    resultat3=result3.select('title','users','ingredients4',bigvect3_('CountVectorizer').alias('CountVectorizer2'))

    x=resultat3.filter(resultat3.title==id).collect()[0].CountVectorizer2
    
    #Calcul de la similarité cosinus entre le vecteur des fréquences des ingrédients du user en entrée de la fonction 
    #et les vecteurs des fréquences des ingrédients de chaque recette

    from math import sqrt

    def prodscal(y):
        return sum(i*j for i,j in zip(x,y))

    def longueur(y):
        return sum(i*i for i in y)

    def cos(y):
        try:
            return float(prodscal(y))/float((sqrt(longueur(y))*sqrt(longueur(x))))
        except:
            return None


    cos_  = F.udf(cos,FloatType())

    resultatcos=resultat3.select('title','users','ingredients4','CountVectorizer2',cos_('CountVectorizer2').alias('cos'))

    resultatcos=resultatcos.filter(resultatcos.title != id)


    for i in recettes3:
        resultatcos=resultatcos.filter(resultatcos.title != i)

    from pyspark.sql.functions import desc

    resultatcos=resultatcos.sort(desc('cos'))
    
    #Recommendation des 5 recettes aux similarités cosinus les plus élevées
    reco=resultatcos.head(5)
    return ([i.title for i in reco])


In [ ]:
#Evaluation du modèle de recommendation de users à users, puis de users à recettes selon la distance de Jaccard 
#pour les individus de la base
dic2={}
for i in identifiants:
    dic2.setdefault(i,recorecettessimicos(i,df2,resultat1))

In [ ]:
dic2

In [ ]:
from pyspark.sql import Row
l = [(x,y) for x,y in zip(dic2.keys(),dic2.values())]
rdd = sc.parallelize(l)
rdd2 = rdd.map(lambda x: Row(uuid2=x[0], recettesreco=x[1]))
dff = sqlContext.createDataFrame(rdd2)

In [ ]:
resultat =resultattest.select('uuid2','recettestest').join(dff,'uuid2')

In [ ]:
#Evaluation de la performance, autrement dit le pourcentage de personnes dont au moins une des 5 recettes recommendées
#appartient aux 1/3 de recettes isolées au début pour chaque individu
def perf(u):
    nb=0;0
    for i in u.recettesreco:
        if i in u.recettestest:
            nb=1.0
            break
    return u.uuid2,u.recettestest,u.recettesreco,nb


l=resultat.rdd.map(lambda row: perf(row)).collect()


In [ ]:
a=[i[3] for i in l]
tauxperfsimicos=float(sum(a))/float(len(a))

In [ ]:
#Calcul de la performance
tauxperfsimicos

In [ ]:
#Recommendations de users à recettes selon la méthode de la similarité cosinus sur les vecteurs des TF-IDF des ingrédients 
def recorecettestfidfcos(id,df,resultat2):
    
    from pyspark.sql.functions import collect_list
    df=df.dropDuplicates()
    grouped_ = df.groupBy(['title']).agg(collect_list("uuid2").alias('users'),collect_list("ingredients").alias('ingredients2'))


    def double(l):
        return list(set(l))


    double_=F.udf(double,ArrayType(StringType()))

    grouped_=grouped_.select('title','users','ingredients2',double_('ingredients2').alias('ingredients3'))

    dfid= resultat2.filter(resultat2.uuid2 ==id )
    recettes3=dfid.collect()[0].listerecettes
    ingredients3=dfid.collect()[0].ingredientsrecettes2

    import re
    from nltk.corpus import stopwords
    from pyspark.ml.feature import StopWordsRemover

    list_nb = '0 1 2 3 4 5 6 7 8 9'.split()
    list_lt='a b c d e f g h i j k l m n o p q r s t u v w kg mg gr cl ml dl'.split()
    stopwords=StopWordsRemover.loadDefaultStopWords('french')

    def transform2 (l):
         return [mot for recette in l for mot in re.split(' |;',''.join([i if i not in list_nb else '' for i in recette ])) if mot!='' if mot not in stopwords and mot not in list_lt] 

    transform2_df  = F.udf(transform2,ArrayType(StringType()))

    grouped_=grouped_.select('title','users','ingredients3',transform2_df('ingredients3').alias('ingredients4'))

    newRow = spark.createDataFrame([[id,[id],ingredients3,ingredients3]])
    grouped_ = grouped_.union(newRow)

    from pyspark.ml.feature import HashingTF, IDF, Tokenizer
    from pyspark.ml.feature import CountVectorizer

    cv = CountVectorizer(inputCol="ingredients4", outputCol="features")
    model = cv.fit(grouped_ )
    results = model.transform(grouped_ )

    #Nous obtenons ici pour chaque individu un vecteur de la taille du corpus des ingrédients des recettes, dont chaque élément 
    #représente le TF-IDF de l'ingrédient dans les recettes consultées par l'individu
    
    idf = IDF(inputCol="features", outputCol="tfidf")
    idfModel = idf.fit(results)
    result2 = idfModel.transform(results)


    from pyspark.mllib.linalg import SparseVector

    def bigvect2(x):
        return ([round(i,3) for i in x])



    bigvect2_  = F.udf(bigvect2,ArrayType(FloatType()))

    resultat2=result2.select('title','userss','ingredients4',bigvect2_('tfidf').alias('tfidf2'))


    x=resultat2.filter(resultat2.title==id).collect()[0].tfidf2
    
    #Calcul de l'indice de la similarité cosinus entre le vecteur des TF-IDF des ingrédients du user en entrée de la fonction 
    #et les vecteurs des TF-IDF des ingrédients de chaque recette

    from math import sqrt

    def prodscal(y):
        return sum(i*j for i,j in zip(x,y))

    def longueur(y):
        return sum(i*i for i in y)

    def cos(y):
        try:
            return float(prodscal(y))/float((sqrt(longueur(y))*sqrt(longueur(x))))
        except:
            return None


    cos_  = F.udf(cos,FloatType())

    resultatcos=resultat2.select('title','users','ingredients4','tfidf2',cos_('tfidf2').alias('cos'))

    resultatcos=resultatcos.filter(resultatcos.title != id)


    for i in recettes3:
        resultatcos=resultatcos.filter(resultatcos.title != i)

    from pyspark.sql.functions import desc

    resultatcos=resultatcos.sort(desc('cos'))

    reco=resultatcos.head(5)
    
    #Recommendations de 5 recettes
    return ([i.title for i in reco])

In [ ]:
dic4={}
for i in identifiants:
    dic4.setdefault(i,recorecettestfidfjac(i,df2,resultat1))

In [ ]:
dic4

In [ ]:
from pyspark.sql import Row
l = [(x,y) for x,y in zip(dic4.keys(),dic4.values())]
rdd = sc.parallelize(l)
rdd2 = rdd.map(lambda x: Row(uuid2=x[0], recettesreco=x[1]))
dff = sqlContext.createDataFrame(rdd2)

In [ ]:
resultat =resultattest.select('uuid2','recettestest').join(dff,'uuid2')

In [ ]:
def perf(u):
    nb=0;0
    for i in u.recettesreco:
        if i in u.recettestest:
            nb=1.0
            break
    return u.uuid2,u.recettestest,u.recettesreco,nb


l=resultat.rdd.map(lambda row: perf(row)).collect()

In [ ]:
a=[i[3] for i in l]
tauxperftfidfcos=float(sum(a))/float(len(a))

In [ ]:
#Calcul de la performance
tauxperftfidfcos

In [ ]:
#Recommendation de users à users, puis de users à recettes selon l'indice de Jaccard sur les vecteurs des fréquences
#des ingrédients
#On cherche les 5 individus les plus proches de l'individu en entrée, et on recommende 5 recettes visionnées par ces individus et 
#différentes de celles visitées par le user en entrée, dans l'ordre de pertinence des individus

def recorecettesjacusers(id,resultat1):
    
    dfid= resultat1.filter(resultat1.uuid2 ==id )
    x=dfid.collect()[0].CountVectorizer2
    recettes1=dfid.collect()[0].listerecettes

    #On calcule l'indice de Jaccard entre le vecteur des fréquences des ingrédients du user en entrée et les autres users de la base train
    def inter(y):
        return sum(i+j for i,j in zip(x,y) if i>0 and j>0)

    def union(y):
        return sum(i+j for i,j in zip(x,y))

    def jaccard(y):
        return round(float(inter(y))/float(union(y)),3)


    jaccard_  = F.udf(jaccard,FloatType())


    resultat_=resultat1.select('uuid2','listerecettes','ingredientsrecettes2','CountVectorizer2',jaccard_('CountVectorizer2').alias('Jaccard'))

    resultat_=resultat_.filter(resultat_.uuid2 != id)


    def remove(l):
        return [i for i in l if i not in recettes1]

    remove_  = F.udf(remove,ArrayType(StringType()))


    resultat_=resultat_.select('uuid2',remove_('listerecettes').alias('listerecettes2'),'ingredientsrecettes2','CountVectorizer2','Jaccard')


    resultatreco=resultat_.dropDuplicates(['listerecettes2'])      


    from pyspark.sql.functions import desc

    resultat_=resultat_.sort(desc('Jaccard'))
    resultatreco=resultatreco.sort(desc('Jaccard'))
    
    reco=[i.listerecettes2 for i in resultatreco.head(5)]
    ingredients=[i.ingredientsrecettes2 for i in resultatreco.head(5)]
    
    reco2=[i.uuid2 for i in resultat_.head(5)]
    
    return ([i for l in reco for i in l ][:5],reco2)

In [ ]:
dic5={}
for i in identifiants:
    dic5.setdefault(i,recorecettesjacusers(i,resultat1))

In [ ]:
dic5

In [ ]:
from pyspark.sql import Row
l = [(x,y) for x,y in zip(dic5.keys(),dic5.values())]
rdd = sc.parallelize(l)
rdd2 = rdd.map(lambda x: Row(uuid2=x[0], recettesreco=x[1][0],usersreco=x[1][1]))
dff = sqlContext.createDataFrame(rdd2)

In [ ]:
dff.show()

In [ ]:
resultat =resultattest.select('uuid2','recettestest').join(dff,'uuid2')

In [ ]:
resultat.show(truncate=False)

In [ ]:
def perf(u):
    nb=0;0
    for i in u.recettesreco:
        if i in u.recettestest:
            nb=1.0
            break
    return u.uuid2,u.recettestest,u.recettesreco,nb,u.usersreco


l=resultat.rdd.map(lambda row: perf(row)).collect()

In [ ]:
l

In [ ]:
#Calcul de la performance
a=[i[3] for i in l]
m=[len(v) for i in l  for v in i[4]]
n=[len(v) for i in l  for v in i[5]]
tauxperfjacusers=float(sum(a))/float(len(a))
nbrecettesmoyenjacusers=float(sum(m))/float(len(m))
nbingredientsmoyenjacusers=float(sum(n))/float(len(n))

In [ ]:
tauxperfjacusers

In [ ]:
nbrecettesmoyenjacusers

In [ ]:
nbingredientsmoyenjacusers

In [ ]:
#Recommendation de users à users, puis de users à recettes selon l'indice de la similarité cosinus sur les vecteurs des fréquences
#des ingrédients
#On cherche les 5 individus les plus proches de l'individu en entrée, et on recommende 5 recettes visionnées par ces individus et 
#différentes de celles visitées par le user en entrée, dans l'ordre de pertinence des individus

from math import sqrt

def recorecettescosusers(id,resultat1):
    
    dfid= resultat1.filter(resultat1.uuid2 ==id )
    x=dfid.collect()[0].CountVectorizer2
    recettes=dfid.collect()[0].listerecettes


    def prodscal(y):
        return sum(i*j for i,j in zip(x,y))

    def longueur(y):
        return sum(i*i for i in y)

    def cos(y):
        try:
            return float(prodscal(y))/float((sqrt(longueur(y))*sqrt(longueur(x))))
        except:
            return None


    cos_  = F.udf(cos,FloatType())

    resultat_=resultat1.select('uuid2','listerecettes','ingredientsrecettes2','CountVectorizer2',cos_('CountVectorizer2').alias('cos'))

    resultat_=resultat_.filter(resultat_.uuid2 != id)
    
    def remove(l):
        return [i for i in l if i not in recettes]

    remove_  = F.udf(remove,ArrayType(StringType()))

    resultat_=resultat_.select('uuid2',remove_('listerecettes').alias('listerecettes2'),'ingredientsrecettes2','CountVectorizer2','cos')

    resultatreco=resultat_.dropDuplicates(['listerecettes2'])  

    from pyspark.sql.functions import desc

    resultat_=resultat_.sort(desc('cos'))
    resultatreco=resultatreco.sort(desc('cos'))
    
    reco=[i.listerecettes2 for i in resultatreco.head(5)]
    
    reco2=[i.uuid2 for i in resultat_.head(5)]
    
    return ([i for l in reco for i in l ][:5],reco2)


In [ ]:
dic6={}
for i in identifiants:
    dic6.setdefault(i,recorecettescosusers(i,resultat1))

In [ ]:
dic6

In [ ]:
from pyspark.sql import Row
l = [(x,y) for x,y in zip(dic6.keys(),dic6.values())]
rdd = sc.parallelize(l)
rdd2 = rdd.map(lambda x: Row(uuid2=x[0], recettesreco=x[1][0],usersreco=x[1][1]))
dff = sqlContext.createDataFrame(rdd2)

In [ ]:
dff.show(truncate=False)

In [ ]:
resultat =resultattest.select('uuid2','recettestest').join(dff,'uuid2')

In [ ]:
resultat.show(truncate=False)

In [ ]:
def perf(u):
    nb=0;0
    for i in u.recettesreco:
        if i in u.recettestest:
            nb=1.0
            break
    return u.uuid2,u.recettestest,u.recettesreco,nb,u.usersreco


l=resultat.rdd.map(lambda row: perf(row)).collect()

In [ ]:
l

In [ ]:
#Calcul de la performance
a=[i[3] for i in l]
m=[len(v) for i in l  for v in i[4]]
n=[len(v) for i in l  for v in i[5]]
tauxperfcosusers=float(sum(a))/float(len(a))
nbrecettesmoyencosusers=float(sum(m))/float(len(m))
nbingredientsmoyencosusers=float(sum(m))/float(len(m))

In [ ]:
tauxperfcosusers

In [ ]:
nbrecettesmoyencosusers

In [ ]:
nbingredientsmoyencosusers

In [ ]:
#Recommendation de users à users, puis de users à recettes selon l'indice de la similarité cosinus sur les vecteurs des TF-IDF
#des ingrédients
#On cherche les 5 individus les plus proches de l'individu en entrée, et on recommende 5 recettes visionnées par ces individus et 
#différentes de celles visitées par le user en entrée, dans l'ordre de pertinence des individus

def recorecettestfidfuserscos(id,resultat2):

    # On fait la recommendation en prenant en entrée un user précis,et en utilisant la similarité cosinus sur les vecteurs des TF-IDF 

    dfid= resultat2.filter(resultat2.uuid2 ==id )
    x=dfid.collect()[0].tfidf2
    recettes2=dfid.collect()[0].listerecettes

    from math import sqrt

    def prodscal(y):
        return sum(i*j for i,j in zip(x,y))

    def longueur(y):
        return sum(i*i for i in y)

    def cos(y):
        try:
            return prodscal(y)/(sqrt(longueur(y))*sqrt(longueur(x)))
        except:
            return None


    cos_  = F.udf(cos,FloatType())

    resultat__=resultat2.select('uuid2','listerecettes','ingredientsrecettes2','tfidf2',cos_('tfidf2').alias('cos'))

    resultat__=resultat__.filter(resultat__.uuid2 != id)

    def remove(l):
        return [i for i in l if i not in recettes2]

    remove_  = F.udf(remove,ArrayType(StringType()))

    resultatreco=resultat__.select('uuid2',remove_('listerecettes').alias('listerecettes2'),'ingredientsrecettes2','tfidf2','cos')

    resultatreco=resultatreco.dropDuplicates(['listerecettes2'])


    from pyspark.sql.functions import desc

    resultat__=resultat__.sort(desc('cos'))
    resultatreco=resultatreco.sort(desc('cos'))

    reco=[i.listerecettes2 for i in resultatreco.head(5)]
    
    reco2=[i.uuid2 for i in resultat_.head(5)]
    
    return ([i for l in reco for i in l ][:5],reco2)




In [ ]:
dic8={}
for i in identifiants:
    dic8.setdefault(i,recorecettestfidfuserscos(i,resultat2))

In [ ]:
dic8

In [ ]:
from pyspark.sql import Row
l = [(x,y) for x,y in zip(dic8.keys(),dic8.values())]
rdd = sc.parallelize(l)
rdd2 = rdd.map(lambda x: Row(uuid2=x[0], recettesreco=x[1][0],usersreco=x[1][1]))
dff = sqlContext.createDataFrame(rdd2)

In [ ]:
dff.show()

In [ ]:
resultat =resultattest.select('uuid2','recettestest').join(dff,'uuid2')

In [ ]:
def perf(u):
    nb=0;0
    for i in u.recettesreco:
        if i in u.recettestest:
            nb=1.0
            break
    return u.uuid2,u.recettestest,u.recettesreco,nb,u.usersreco


l=resultat.rdd.map(lambda row: perf(row)).collect()

In [ ]:
#Calcul de la performance
a=[i[3] for i in l]
tauxperftfidfuserscos=float(sum(a))/float(len(a))

In [ ]:
tauxperftfidfuserscos

In [ ]:
#On cherche à savoir quelles sont les 5 recettes les plus vues que l'on recommendera à tous les individus
from pyspark.sql.functions import collect_list
df2=df2.dropDuplicates()
grouped_ = df.groupBy(['title']).agg(collect_list("uuid2").alias('users'),collect_list("ingredients").alias('ingredients2'))

def count(u):
    return float(len(u))

count_=F.udf(count,FloatType())


groupedmostseen = grouped_ .select('users','ingredients2','title',count_('users').alias('visites'))
groupedmostseen= groupedmostseen.sort(desc('visites'))
groupedmostseen.show()

In [ ]:
recomostseen=[ i.title for i in groupedmostseen.select('title').head(5)]

In [ ]:
dic5={}
for i in identifiants:
    dic5.setdefault(i,recomostseen)

In [ ]:
from pyspark.sql import Row
l = [(x,y) for x,y in zip(dic5.keys(),dic5.values())]
rdd = sc.parallelize(l)
rdd2 = rdd.map(lambda x: Row(uuid2=x[0], recettesreco=x[1]))
dff = sqlContext.createDataFrame(rdd2)

In [ ]:
resultat =resultattest.select('uuid2','recettestest').join(dff,'uuid2')

In [ ]:
def perf(u):
    nb=0;0
    for i in u.recettesreco:
        if i in u.recettestest:
            nb=1.0
            break
    return u.uuid2,u.recettestest,u.recettesreco,nb


l=resultat.rdd.map(lambda row: perf(row)).collect()

In [ ]:
#Calcul de la performance
a=[i[3] for i in l]
tauxperfrecorecettesmostseen=float(sum(a))/float(len(a))

In [ ]:
tauxperfrecorecettesmostseen